In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.feature_selection import SelectFromModel,SequentialFeatureSelector

from sklearn.metrics import f1_score,accuracy_score,recall_score,precision_score,confusion_matrix,roc_auc_score

import warnings
warnings.simplefilter('ignore')

In [2]:
def model_basic(x_train, y_train, x_test, y_test):
    models = [
        LogisticRegression(),
        LinearDiscriminantAnalysis(),
        GaussianNB(),
        RandomForestClassifier(),
        XGBClassifier(),
        
    ]

    rdict = {'model': [], 'accuracy': [], 'precision': [], 'recall': [], 'f1_score': [], 'auc_score': []}

    for clf in models:
        clf = clf.fit(x_train, y_train)
        pred = clf.predict(x_test)
        pred_prob_rf = clf.predict_proba(x_test)[:, 1].reshape(-1, 1)
        auc_score = roc_auc_score(y_test, pred_prob_rf)
        results = (
            round(accuracy_score(y_test, pred), 4),
            round(precision_score(y_test, pred), 4),
            round(recall_score(y_test, pred), 4),
            round(f1_score(y_test, pred), 4),
            round(auc_score, 4)
        )

        rdict['model'].append(clf)
        rdict['accuracy'].append(results[0])
        rdict['precision'].append(results[1])
        rdict['recall'].append(results[2])
        rdict['f1_score'].append(results[3])
        rdict['auc_score'].append(results[4])

        confusion = confusion_matrix(y_test, pred)
        print(confusion)

    rdf = pd.DataFrame(data=rdict)
    return rdf

In [3]:
train = pd.read_csv("./datasets/통계검증완료/코스닥_이상치+통계검증_train.csv",index_col=0)
test =  pd.read_csv('./datasets/통계검증완료/코스닥_이상치+통계검증_test.csv',index_col=0)



In [4]:
print(test["target_1"].value_counts())
print(test["target_2"].value_counts())
print(test["target_3"].value_counts())

0    2817
1      75
Name: target_1, dtype: int64
0    2819
1      73
Name: target_2, dtype: int64
0    2820
1      72
Name: target_3, dtype: int64


---
# 산업별 더미변수 - 제조업,건설업,정보통신업,그외
---

In [5]:
# train.loc[~(train['산업군'].str.contains('제조업') | train['산업군'].str.contains('건설업') | train['산업군'].str.contains('정보통신업')), '산업군'] = '그 외'
# dummy_df = pd.get_dummies(train['산업군'])
# train = pd.concat([train,dummy_df],axis=1)

In [6]:
# test.loc[~(test['산업군'].str.contains('제조업') | test['산업군'].str.contains('건설업') | test['산업군'].str.contains('정보통신업')), '산업군'] = '그 외'
# dummy_df = pd.get_dummies(test['산업군'])
# test = pd.concat([test,dummy_df],axis=1)

----

In [7]:
train["회계년도"] = train["회계년도"].astype("object")
train["거래소코드"] = train["거래소코드"].astype("object")
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5014 entries, 0 to 5013
Data columns (total 35 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   회사명                    5014 non-null   object 
 1   거래소코드                  5014 non-null   object 
 2   회계년도                   5014 non-null   object 
 3   산업군                    5014 non-null   object 
 4   총자본순이익률                5014 non-null   float64
 5   EBIT/총자산               5014 non-null   float64
 6   총자본정상영업이익률             5014 non-null   float64
 7   대주주지분율                 5014 non-null   float64
 8   타인자본회전률                5014 non-null   float64
 9   차입금의존도                 5014 non-null   float64
 10  현금흐름 대 자산              5014 non-null   float64
 11  현금흐름/총부채비율             5014 non-null   float64
 12  순운전자본비율                5014 non-null   float64
 13  현금흐름 대 매출액             5014 non-null   float64
 14  EBITDA마진율              5014 non-null   float64
 15  매출액총

In [8]:
col_int = train.select_dtypes("float").columns
col_int

Index(['총자본순이익률', 'EBIT/총자산', '총자본정상영업이익률', '대주주지분율', '타인자본회전률', '차입금의존도',
       '현금흐름 대 자산', '현금흐름/총부채비율', '순운전자본비율', '현금흐름 대 매출액', 'EBITDA마진율',
       '매출액총이익률', '누적수익성비율', 'abs(영업현금흐름-당기순이익)/매출액', '자본금회전률', '당기전기영업손익',
       '금융비용부담률', '영업현금흐름-단기차입금', 'TMD', '총자본증가율', 'FINDEP', '외국인지분율',
       '자기자본증가율', '매출액증가율', '정상영업이익증가율', '자기자본순이익률', 'ROA변화율', '이자부담률'],
      dtype='object')

---
#target1
---

In [9]:
x_train = train[col_int]
y_train = train[['target_1']]

x_test = test[col_int]
y_test = test[['target_1']]

In [10]:
y_test.value_counts()

target_1
0           2817
1             75
dtype: int64

In [11]:
#  선정된 피쳐를 저장하기 위한 데이터프레임 생성 
df_select = pd.DataFrame()
# 생성된 데이터프레임의 인덱스 지정
df_select.index = x_train.columns

---
# lasso - 임베디드기법
---

In [12]:
#  lasso
selector = SelectFromModel(estimator=LogisticRegression(penalty='l1',solver='liblinear',C=0.01)).fit(x_train, y_train)
df_select["lasso_0.01"] = selector.get_support()

selector = SelectFromModel(estimator=LogisticRegression(penalty='l1',solver='liblinear',C=0.05)).fit(x_train, y_train)
df_select["lasso_0.05"] = selector.get_support()

selector = SelectFromModel(estimator=LogisticRegression(penalty='l1',solver='liblinear',C=0.1)).fit(x_train, y_train)
df_select["lasso_0.1"] = selector.get_support()


---
# stepwise - 래퍼기법
---

In [13]:
# logit foward
estimator = LogisticRegression(penalty='l1',solver='liblinear',C=0.05)
selector = SequentialFeatureSelector(estimator,direction='forward',cv=5,n_jobs=-1)
selector.fit(x_train, y_train)
df_select['logit_fwd'] = selector.get_support().tolist()
# logit backward
selector = SequentialFeatureSelector(estimator,direction='backward',cv=5,n_jobs=-1)
selector.fit(x_train, y_train)
df_select['logit_bwd'] = selector.get_support().tolist()

In [ ]:
# lda foward
estimator = LinearDiscriminantAnalysis(solver='svd')
selector = SequentialFeatureSelector(estimator,direction='forward',cv=5,n_jobs=-1)
selector.fit(x_train, y_train)
df_select['lda_fwd'] = selector.get_support().tolist()
# lda backward
selector = SequentialFeatureSelector(estimator,direction='backward',cv=5,n_jobs=-1,)
selector.fit(x_train, y_train)
df_select['lda_bwd'] = selector.get_support().tolist()

---
## 비모수적 방법
---

In [ ]:
# # RandomForest_foward
# estimator = RandomForestClassifier()
# selector = SequentialFeatureSelector(estimator,direction='forward',cv=2,n_jobs=-1)
# selector.fit(x_train, y_train)
# df_select['RFC_fwd'] = selector.get_support().tolist()
# # RandomForest _ backward
# selector = SequentialFeatureSelector(estimator,direction='backward',cv=2,n_jobs=-1)
# selector.fit(x_train, y_train)
# df_select['RFC_bwd'] = selector.get_support().tolist()

---
## 최종 select
---

In [ ]:
for col in df_select.columns.to_list():
    df_select[col] = df_select[col].apply(lambda x : 1 if x==True else 0)
df_select['total'] = df_select.sum(axis=1)
df_select[df_select['total']>=5].index

Index(['대주주지분율', '순운전자본비율', 'EBITDA마진율', '누적수익성비율', '자본금회전률', 'TMD', '총자본증가율',
       '자기자본증가율', '정상영업이익증가율', '자기자본순이익률', 'ROA변화율', '이자부담률'],
      dtype='object')

In [ ]:
df_select[df_select['total']>=5]

,lasso_0.01,lasso_0.05,lasso_0.1,logit_fwd,logit_bwd,lda_fwd,lda_bwd,total
대주주지분율,1,1,1,0,0,1,1,5
순운전자본비율,1,1,1,1,1,0,0,5
EBITDA마진율,1,1,1,1,0,1,1,6
누적수익성비율,0,1,1,0,1,1,1,5
자본금회전률,1,1,1,1,1,1,1,7
TMD,1,1,1,1,1,1,1,7
총자본증가율,0,1,1,0,1,1,1,5
자기자본증가율,1,1,1,1,1,0,0,5
정상영업이익증가율,1,1,1,1,1,0,1,6
자기자본순이익률,1,1,1,1,1,1,0,6


In [ ]:
최종_col_1 =df_select[df_select['total']>=5].index.to_list()

# temp = ['EBIT/총자산', 'TMD', 'EBITDA마진율', '누적수익성비율', 'abs(영업현금흐름-당기순이익)/매출액', '순운전자본비율','당기전기영업손익']
x_train_1 = x_train[최종_col_1]
x_test_1 = x_test[최종_col_1]

In [ ]:
model_basic(x_train_1,y_train,x_train_1,y_train)

[[4808   34]
 [ 105   67]]
[[4811   31]
 [ 100   72]]
[[4621  221]
 [  56  116]]
[[4842    0]
 [   0  172]]
[[4842    0]
 [   0  172]]


,model,accuracy,precision,recall,f1_score,auc_score
0,LogisticRegression(),0.9723,0.6634,0.3895,0.4908,0.8935
1,LinearDiscriminantAnalysis(),0.9739,0.6990,0.4186,0.5236,0.9125
2,GaussianNB(),0.9448,0.3442,0.6744,0.4558,0.9089
3,"(DecisionTreeClassifier(max_features='sqrt', r...",1.0000,1.0000,1.0000,1.0000,1.0000
4,"XGBClassifier(base_score=None, booster=None, c...",1.0000,1.0000,1.0000,1.0000,1.0000


In [ ]:
model_basic(x_train_1,y_train,x_test_1,y_test)

[[2781   36]
 [  36   39]]
[[2775   42]
 [  29   46]]
[[2665  152]
 [  13   62]]
[[2806   11]
 [  27   48]]
[[2801   16]
 [  28   47]]


,model,accuracy,precision,recall,f1_score,auc_score
0,LogisticRegression(),0.9751,0.5200,0.5200,0.5200,0.9114
1,LinearDiscriminantAnalysis(),0.9754,0.5227,0.6133,0.5644,0.9097
2,GaussianNB(),0.9429,0.2897,0.8267,0.4291,0.9674
3,"(DecisionTreeClassifier(max_features='sqrt', r...",0.9869,0.8136,0.6400,0.7164,0.9677
4,"XGBClassifier(base_score=None, booster=None, c...",0.9848,0.7460,0.6267,0.6812,0.9600


-----
# target2

In [ ]:
x_train = train[col_int]
y_train = train[['target_2']]

x_test = test[col_int]
y_test = test[['target_2']]

#  선정된 피쳐를 저장하기 위한 데이터프레임 생성 
df_select_2 = pd.DataFrame()
# 생성된 데이터프레임의 인덱스 지정
df_select_2.index = x_train.columns

In [ ]:
#  lasso
selector = SelectFromModel(estimator=LogisticRegression(penalty='l1',solver='liblinear',C=0.01)).fit(x_train, y_train)
df_select_2["lasso_0.01"] = selector.get_support()

selector = SelectFromModel(estimator=LogisticRegression(penalty='l1',solver='liblinear',C=0.05)).fit(x_train, y_train)
df_select_2["lasso_0.05"] = selector.get_support()

selector = SelectFromModel(estimator=LogisticRegression(penalty='l1',solver='liblinear',C=0.1)).fit(x_train, y_train)
df_select_2["lasso_0.1"] = selector.get_support()

# logit foward
estimator = LogisticRegression(penalty='l1',solver='liblinear',C=0.05)
selector = SequentialFeatureSelector(estimator,direction='forward',cv=5,n_jobs=-1)
selector.fit(x_train, y_train)
df_select_2['logit_fwd'] = selector.get_support().tolist()
# logit backward
selector = SequentialFeatureSelector(estimator,direction='backward',cv=5,n_jobs=-1)
selector.fit(x_train, y_train)
df_select_2['logit_bwd'] = selector.get_support().tolist()


# lda foward
estimator = LinearDiscriminantAnalysis(solver='svd')
selector = SequentialFeatureSelector(estimator,direction='forward',cv=5,n_jobs=-1)
selector.fit(x_train, y_train)
df_select_2['lda_fwd'] = selector.get_support().tolist()
# lda backward
selector = SequentialFeatureSelector(estimator,direction='backward',cv=5,n_jobs=-1,)
selector.fit(x_train, y_train)
df_select_2['lda_bwd'] = selector.get_support().tolist()

# # RandomForest_foward
# estimator = RandomForestClassifier()
# selector = SequentialFeatureSelector(estimator,direction='forward',cv=2,n_jobs=-1)
# selector.fit(x_train, y_train)
# df_select_2['RFC_fwd'] = selector.get_support().tolist()
# # RandomForest _ backward
# selector = SequentialFeatureSelector(estimator,direction='backward',cv=2,n_jobs=-1)
# selector.fit(x_train, y_train)
# df_select_2['RFC_bwd'] = selector.get_support().tolist()

In [23]:
for col in df_select_2.columns.to_list():
    df_select_2[col] = df_select_2[col].apply(lambda x : 1 if x==True else 0)
df_select_2['total'] = df_select_2.sum(axis=1)
df_select_2[df_select_2['total']>=5].index


Index(['대주주지분율', '순운전자본비율', '매출액총이익률', '누적수익성비율', '자본금회전률', '당기전기영업손익',
       '금융비용부담률', 'TMD', '정상영업이익증가율', '자기자본순이익률', '이자부담률'],
      dtype='object')

In [24]:
df_select_2[df_select_2['total']>=5]

,lasso_0.01,lasso_0.05,lasso_0.1,logit_fwd,logit_bwd,lda_fwd,lda_bwd,total
대주주지분율,1,1,1,0,1,0,1,5
순운전자본비율,1,1,1,1,1,1,1,7
매출액총이익률,1,1,1,0,1,1,0,5
누적수익성비율,0,1,1,1,1,0,1,5
자본금회전률,1,1,1,0,1,1,0,5
당기전기영업손익,1,1,1,1,0,1,1,6
금융비용부담률,1,1,1,1,1,0,0,5
TMD,1,1,1,1,1,1,1,7
정상영업이익증가율,1,1,1,1,1,1,1,7
자기자본순이익률,1,1,1,1,1,1,1,7


In [25]:
최종_col =df_select_2[df_select_2['total']>=5].index.to_list()

x_train_2 = x_train[최종_col]
x_test_2 = x_test[최종_col]

In [26]:
model_basic(x_train_2,y_train,x_train_2,y_train)

[[4771   32]
 [ 144   67]]
[[4761   42]
 [ 138   73]]
[[4653  150]
 [  93  118]]
[[4803    0]
 [   0  211]]
[[4803    0]
 [   0  211]]


,model,accuracy,precision,recall,f1_score,auc_score
0,LogisticRegression(),0.9649,0.6768,0.3175,0.4323,0.8942
1,LinearDiscriminantAnalysis(),0.9641,0.6348,0.3460,0.4479,0.9037
2,GaussianNB(),0.9515,0.4403,0.5592,0.4927,0.8903
3,"(DecisionTreeClassifier(max_features='sqrt', r...",1.0000,1.0000,1.0000,1.0000,1.0000
4,"XGBClassifier(base_score=None, booster=None, c...",1.0000,1.0000,1.0000,1.0000,1.0000


In [27]:
model_basic(x_train_2,y_train,x_test_2,y_test)

[[2785   34]
 [  40   33]]
[[2769   50]
 [  23   50]]
[[2684  135]
 [  17   56]]
[[2803   16]
 [  27   46]]
[[2791   28]
 [  29   44]]


,model,accuracy,precision,recall,f1_score,auc_score
0,LogisticRegression(),0.9744,0.4925,0.4521,0.4714,0.8873
1,LinearDiscriminantAnalysis(),0.9748,0.5000,0.6849,0.5780,0.9678
2,GaussianNB(),0.9474,0.2932,0.7671,0.4242,0.9700
3,"(DecisionTreeClassifier(max_features='sqrt', r...",0.9851,0.7419,0.6301,0.6815,0.9844
4,"XGBClassifier(base_score=None, booster=None, c...",0.9803,0.6111,0.6027,0.6069,0.9730


----
# target3

In [28]:
x_train = train[col_int]
y_train = train[['target_3']]

x_test = test[col_int]
y_test = test[['target_3']]

#  선정된 피쳐를 저장하기 위한 데이터프레임 생성 
df_select_3 = pd.DataFrame()
# 생성된 데이터프레임의 인덱스 지정
df_select_3.index = x_train.columns

In [29]:
#  lasso
selector = SelectFromModel(estimator=LogisticRegression(penalty='l1',solver='liblinear',C=0.01)).fit(x_train, y_train)
df_select_3["lasso_0.01"] = selector.get_support()

selector = SelectFromModel(estimator=LogisticRegression(penalty='l1',solver='liblinear',C=0.05)).fit(x_train, y_train)
df_select_3["lasso_0.05"] = selector.get_support()

selector = SelectFromModel(estimator=LogisticRegression(penalty='l1',solver='liblinear',C=0.1)).fit(x_train, y_train)
df_select_3["lasso_0.1"] = selector.get_support()

# logit foward
estimator = LogisticRegression(penalty='l1',solver='liblinear',C=0.05)
selector = SequentialFeatureSelector(estimator,direction='forward',cv=5,n_jobs=-1)
selector.fit(x_train, y_train)
df_select_3['logit_fwd'] = selector.get_support().tolist()
# logit backward
selector = SequentialFeatureSelector(estimator,direction='backward',cv=5,n_jobs=-1)
selector.fit(x_train, y_train)
df_select_3['logit_bwd'] = selector.get_support().tolist()


# lda foward
estimator = LinearDiscriminantAnalysis(solver='svd')
selector = SequentialFeatureSelector(estimator,direction='forward',cv=5,n_jobs=-1)
selector.fit(x_train, y_train)
df_select_3['lda_fwd'] = selector.get_support().tolist()
# lda backward
selector = SequentialFeatureSelector(estimator,direction='backward',cv=5,n_jobs=-1,)
selector.fit(x_train, y_train)
df_select_3['lda_bwd'] = selector.get_support().tolist()

# # RandomForest_foward
# estimator = RandomForestClassifier()
# selector = SequentialFeatureSelector(estimator,direction='forward',cv=2,n_jobs=-1)
# selector.fit(x_train, y_train)
# df_select_3['RFC_fwd'] = selector.get_support().tolist()
# # RandomForest _ backward
# selector = SequentialFeatureSelector(estimator,direction='backward',cv=2,n_jobs=-1)
# selector.fit(x_train, y_train)
# df_select_3['RFC_bwd'] = selector.get_support().tolist()

In [30]:
for col in df_select_3.columns.to_list():
    df_select_3[col] = df_select_3[col].apply(lambda x : 1 if x==True else 0)
df_select_3['total'] = df_select_3.sum(axis=1)
df_select_3[df_select_3['total']>=5].index

Index(['대주주지분율', '타인자본회전률', '차입금의존도', '순운전자본비율', 'EBITDA마진율', '매출액총이익률',
       '누적수익성비율', '자본금회전률', '당기전기영업손익', '금융비용부담률', 'TMD', '총자본증가율',
       '정상영업이익증가율', '자기자본순이익률', 'ROA변화율'],
      dtype='object')

In [31]:
df_select_3[df_select_3['total']>=5]

,lasso_0.01,lasso_0.05,lasso_0.1,logit_fwd,logit_bwd,lda_fwd,lda_bwd,total
대주주지분율,1,1,1,0,1,1,0,5
타인자본회전률,0,1,1,0,1,1,1,5
차입금의존도,1,1,1,1,1,0,0,5
순운전자본비율,1,1,1,0,1,0,1,5
EBITDA마진율,1,1,1,0,1,0,1,5
매출액총이익률,1,1,1,1,1,0,0,5
누적수익성비율,0,1,1,1,1,1,1,6
자본금회전률,1,1,1,1,1,1,1,7
당기전기영업손익,1,1,1,1,0,1,0,5
금융비용부담률,1,1,1,0,1,0,1,5


In [32]:
최종_col =df_select_3[df_select_3['total']>=5].index.to_list()

x_train_3 = x_train[최종_col]
x_test_3 = x_test[최종_col]

In [33]:
model_basic(x_train_3,y_train,x_train_3,y_train)

[[4823   26]
 [  98   67]]
[[4820   29]
 [  93   72]]
[[4684  165]
 [  59  106]]
[[4849    0]
 [   0  165]]
[[4849    0]
 [   0  165]]


,model,accuracy,precision,recall,f1_score,auc_score
0,LogisticRegression(),0.9753,0.7204,0.4061,0.5194,0.9173
1,LinearDiscriminantAnalysis(),0.9757,0.7129,0.4364,0.5414,0.9181
2,GaussianNB(),0.9553,0.3911,0.6424,0.4862,0.9258
3,"(DecisionTreeClassifier(max_features='sqrt', r...",1.0000,1.0000,1.0000,1.0000,1.0000
4,"XGBClassifier(base_score=None, booster=None, c...",1.0000,1.0000,1.0000,1.0000,1.0000


In [34]:
model_basic(x_train_3,y_train,x_test_3,y_test)

[[2793   27]
 [  36   36]]
[[2771   49]
 [  23   49]]
[[2692  128]
 [  17   55]]
[[2809   11]
 [  24   48]]
[[2805   15]
 [  25   47]]


,model,accuracy,precision,recall,f1_score,auc_score
0,LogisticRegression(),0.9782,0.5714,0.5000,0.5333,0.8986
1,LinearDiscriminantAnalysis(),0.9751,0.5000,0.6806,0.5765,0.9529
2,GaussianNB(),0.9499,0.3005,0.7639,0.4314,0.9677
3,"(DecisionTreeClassifier(max_features='sqrt', r...",0.9879,0.8136,0.6667,0.7328,0.9816
4,"XGBClassifier(base_score=None, booster=None, c...",0.9862,0.7581,0.6528,0.7015,0.9810


In [35]:
x_train = train[col_int]
y_train = train[['target_3']]

x_test = test[col_int]
y_test = test[['target_3']]

최종_col = ['자기자본순이익률', 'EBITDA마진율', '누적수익성비율', '당기전기영업손익', '총자본증가율',
       'abs(영업현금흐름-당기순이익)/매출액', 'TMD']

x_train_3 = x_train[최종_col]
x_test_3 = x_test[최종_col]

model_basic(x_train_3,y_train,x_test_3,y_test)

[[2720  100]
 [  25   47]]
[[2781   39]
 [  25   47]]
[[2722   98]
 [  18   54]]
[[2800   20]
 [  27   45]]
[[2796   24]
 [  28   44]]


,model,accuracy,precision,recall,f1_score,auc_score
0,LogisticRegression(),0.9568,0.3197,0.6528,0.4292,0.9119
1,LinearDiscriminantAnalysis(),0.9779,0.5465,0.6528,0.5949,0.9667
2,GaussianNB(),0.9599,0.3553,0.7500,0.4821,0.9625
3,"(DecisionTreeClassifier(max_features='sqrt', r...",0.9837,0.6923,0.6250,0.6569,0.9512
4,"XGBClassifier(base_score=None, booster=None, c...",0.9820,0.6471,0.6111,0.6286,0.9380


In [36]:
print("최종_col_1 : ",x_train_1.columns)
print("최종_col_2 : ",x_train_2.columns)
print("최종_col_3 : ",x_train_3.columns)


최종_col_1 :  Index(['대주주지분율', '순운전자본비율', 'EBITDA마진율', '누적수익성비율', '자본금회전률', 'TMD', '총자본증가율',
       '자기자본증가율', '정상영업이익증가율', '자기자본순이익률', 'ROA변화율', '이자부담률'],
      dtype='object')
최종_col_2 :  Index(['대주주지분율', '순운전자본비율', '매출액총이익률', '누적수익성비율', '자본금회전률', '당기전기영업손익',
       '금융비용부담률', 'TMD', '정상영업이익증가율', '자기자본순이익률', '이자부담률'],
      dtype='object')
최종_col_3 :  Index(['자기자본순이익률', 'EBITDA마진율', '누적수익성비율', '당기전기영업손익', '총자본증가율',
       'abs(영업현금흐름-당기순이익)/매출액', 'TMD'],
      dtype='object')
